In [ ]:
sys.executable

In [1]:
sys.path
# sys.executable

['/Users/katiestutts/code/cnn_practice',
 '/Users/katiestutts/.vscode/extensions/ms-toolsai.jupyter-2020.12.414227025/pythonFiles',
 '/Users/katiestutts/.vscode/extensions/ms-toolsai.jupyter-2020.12.414227025/pythonFiles/lib/python',
 '/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.8/lib/python38.zip',
 '/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8',
 '/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8/lib-dynload',
 '',
 '/Users/katiestutts/Library/Python/3.8/lib/python/site-packages',
 '/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8/site-packages',
 '/Users/katiestutts/Library/Python/3.8/lib/python/site-packages/IPython/extensions',
 '/Users/katiestutts/.ipython']

In [7]:
import os
import librosa 
import math
import json

DATASET_PATH="Data_Reduced"
# print(DATASET_PATH)
JSON_PATH="data.json"
# print(JSON_PATH)
SAMPLE_RATE=22050 # customary value for sr in music processing
DURATION=30 # 30 seconds per audio file
SAMPLES_PER_TRACK=SAMPLE_RATE * DURATION

def save_mfcc(dataset_path, json_path, n_mfcc=13, n_fft=2048, hop_length=512, num_segments=5): # num segments is chopping up each track into a number of different segments
    print("heello")
    # build a dict to store data
    data = {
        "mapping": [], # need a way of mapping different genre labels onto numbers
        "mfcc": [], # is the training data, the training inputs
        "labels": [] # the outputs or targets (genres) we are expecting
    }

    # samples_per_track is the overall number of samples in a track..
    # number of samples for EACH segment
    num_samples_per_segment = int(SAMPLES_PER_TRACK / num_segments)
    # we are calculating the mfccs at each hop length
    expected_num_mfcc_vectors_per_segment = math.ceil(num_samples_per_segment / hop_length) # this could potentially becme a float but we want to round to ceil
    # print("right before for loop")
    # root_dir = "Test"
    # # print(root_dir)
    # file_list = []
    # path_list = []

    # for subdir, dirs, files in os.walk(root_dir):
    #     print(root_dir)
    #     for file in files:
    #         path_list.append(os.path.join(subdir, file))
    #         file_list.append(file)
        
    # print(path_list)
    # print(file_list)
    # loop through all of the genres
    i = 0
    for dirpath, dirnames, filenames in os.walk(dataset_path):
        i +=1
        print("inside for loop")
        # ensure that we are not at the root level (the dataset label)
        # aka we wanna make sure were at the genre subfolder level
        if dirpath is not dataset_path:
            # save the semantic label (the genre)
            dirpath_components = dirpath.split("/") # genre/blues => ["genre", "blues"]
            semantic_label = dirpath_components[-1]
            data["mapping"].append(semantic_label)
            print("\nProcessing: {}".format(semantic_label))

            # process files for a specfic genre
            for file in filenames: 
                # load the audio file
                file_path = os.path.join(dirpath, file) 
                signal, sr = librosa.load(file_path, sr=SAMPLE_RATE)

                # cannot extract mfcc at this level. need to extract that at the segment level
                # process segments extracting mfcc and storing data GO ONE LEVEL DEEPER
                for segment in range(num_segments):
                    # for segment = 0 (first segment), this is equal to 0 because we are starting at 0
                    start_sample = num_samples_per_segment * segment 
                    # for segment = 0, this is equal to the num_of_samples_per_segment
                    # from here, we continually slide to the right for each segment we are calculating
                    finish_sample = start_sample + num_samples_per_segment

                    # extract out an mfcc of the signal, but only during a specified segment
                    mfcc = librosa.feature.mfcc(signal[start_sample:finish_sample], sr=sr, n_fft=n_fft, n_mfcc=n_mfcc, hop_length=hop_length) # these are from args
                    mfcc = mfcc.T

                    # we want all training data to have all of the same shape
                    # we want to store mfcc for segment if it has the expected length
                    if len(mfcc) == expected_num_mfcc_vectors_per_segment:
                        data["mfcc"].append(mfcc.tolist())
                        data["labels"].append(i-1) 
                        print("{}, segment:{}".format(file_path, segment))

            print("goodbyee")
            with open(json_path, "w") as fp:
                json.dump(data, fp, indent=4)


if __name__ == "__main__":
    save_mfcc(DATASET_PATH, JSON_PATH, num_segments=10)

ModuleNotFoundError: No module named 'librosa'